![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/annotation/text/english/stemmer/Word_Stemming_with_Stemmer.ipynb)


# **Word Stemming with Stemmer**

In these examples we to stem words with Stemmer.

## **0. Colab Setup**

Only run this block if you are inside Google Colab to set up Spark NLP otherwise
skip it.

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [ ]:
from pyspark.ml import Pipeline

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

Spark NLP version:  4.3.1
Apache Spark version:  3.3.0


### **Create Spark Dataframe**

In [ ]:
spark_df = spark.read.text('../spark-nlp-basics/sample-sentences-en.txt').toDF('text')

spark_df.show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



### **Transformers**

## Stemmer

Returns hard-stems out of words with the objective of retrieving the meaningful part of the word


In [ ]:
stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[documentAssembler,
                               tokenizer,
                               stemmer])

result = nlpPipeline.fit(spark_df).transform(spark_df)

result.show(truncate=40)

+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|                                    text|                                document|                                   token|                                    stem|
+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|            Peter is a very good person.|[{document, 0, 27, Peter is a very go...|[{token, 0, 4, Peter, {sentence -> 0}...|[{token, 0, 4, peter, {sentence -> 0}...|
|  My life in Russia is very interesting.|[{document, 0, 37, My life in Russia ...|[{token, 0, 1, My, {sentence -> 0}, [...|[{token, 0, 1, my, {sentence -> 0}, [...|
|John and Peter are brothers. However ...|[{document, 0, 76, John and Peter are...|[{token, 0, 3, John, {sentence -> 0},...|[{token, 0, 3, john, {sentence -> 0},...|
|Luc

In [ ]:
result.select('stem.result').show(truncate=False)

+-------------------------------------------------------------------------------------------+
|result                                                                                     |
+-------------------------------------------------------------------------------------------+
|[peter, i, a, veri, good, person, .]                                                       |
|[my, life, in, russia, i, veri, interest, .]                                               |
|[john, and, peter, ar, brother, ., howev, thei, don't, support, each, other, that, much, .]|
|[luca, nogal, dunberck, i, no, longer, happi, ., he, ha, a, good, car, though, .]          |
|[europ, i, veri, cultur, rich, ., there, ar, huge, church, !, and, big, hous, !]           |
+-------------------------------------------------------------------------------------------+



If you are using PySpark 3.1.2 or below, You should use this format.

```python
import pyspark.sql.functions as F

result_df = result.select(F.explode(F.arrays_zip("token.result", "stem.result")).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("stem")).toPandas()

result_df.head(10)
```

In [ ]:
import pyspark.sql.functions as F

result_df = result.select(F.explode(F.arrays_zip(result.token.result,
                                                 result.stem.result)).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("stem")).toPandas()

result_df.head(10)

,token,stem
0,Peter,peter
1,is,i
2,a,a
3,very,veri
4,good,good
5,person,person
6,.,.
7,My,my
8,life,life
9,in,in
